In [384]:
# para buscar archivos
from glob import glob
# super excel para analizar
import pandas as pd
# para leer los datos .json
import json
# para hacer busquedas en strings
import re
# para operaciones numericas
import numpy as np
# para que quede mas lindo
import warnings
warnings.filterwarnings('ignore')

In [385]:
# Para bajar los datos (Mati)
# aws s3 sync s3://unket/gyani ./data

In [465]:
def dato2df(f):
    """recibe path al archivo y devuelve DataFrame"""
    with open(f, 'r') as inp:
        i = inp.read()#.decode('latin1')
        #i = re.sub(r"\\+", r"", i)
        i = json.loads(i)[1:-1]
        #i = re.sub(r'"{', r'\"{', i)
        #i = re.sub(r'}"', r'}\"', i)
    return i#pd.read_json(j)

In [466]:
files = glob('./datos/*.json')
len(files)

12

In [467]:
data = []
for f in files:
    d = dato2df(f)
    data.append(d)    
    break

In [468]:
print(d)

[{\"rt\":2016,\"stimulus\":\"<font size=5.5><p>¡Hola!</p><p><p>Esta encuesta forma parte del proyecto de investigación y divulgación científica <b>Neurotransmitiendo</b>.</p><p>La encuesta está dirigida únicamente a psicólogos recibidos que estén o hayan trabajado en el ámbito clínico. Se agradece mucho la difusión de la misma a colegas que trabajen en el mismo ámbito.</p><p>La participación en la misma es voluntaria teniendo el control de desistir si lo desea. La información que se recoja con este instrumento es confidencial y se utilizará solamente con fines científicos. Al participar de esta encuesta en línea su consentimiento informado será tomado como dado. Ante cualquier duda, puede escribir a <b>ntransmitiendo@gmail.com</b>\",\"button_pressed\":\"0\",\"trial_type\":\"html-button-response\",\"trial_index\":0,\"time_elapsed\":2032,\"internal_node_id\":\"0.0-0.0\"},{\"rt\":3326,\"stimulus\":\"<font size=5.5><p>Continuando con el experimento aceptás aportar los datos generados, ayud

In [469]:
# No pude parsear el json
json.loads(d)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [397]:
df = pd.DataFrame(data)

In [398]:
df.shape

(1, 1)